In [11]:
from skimage import io, feature
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.color import label2rgb
import skimage.filters as sk
import skimage.filters as filters
from skimage.morphology import erosion, dilation, disk
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import math

res = [] # Массив результатов
k = 0 # Число найденных треугольников

img = io.imread("C:\\Users\\Frederik\\Desktop\\Pict_2_2.bmp", as_gray=True) # Считываем изображение в сером цвете
bw = feature.canny(img, sigma=3) # Применяем к полученному изображению оператор Кэннни с параметром сглаживания, равным 3
cleared = clear_border(bw) 
label_image = label(cleared) # Отделяем фон от полученного изображения
for region in regionprops(label_image): # Начинаем обработку каждого из найденного контура
    if 550 >= region.area >= 150: # Из-за того, что мы знаем сторону треугольника, определяем площадь возможных областей, чтобы не выбрать то, что нам не нужно
        k += 1 # Увеличиваем число найденных треугольников на 1
        minr, minc, maxr, maxc = region.bbox # Определяем параметры для изображения найденных областей
        rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr, # Строим полученные прямоугольники на изображении
                                  fill=False, edgecolor='red', linewidth=2)
        
cr = [] # Массив вырезанных изображений, содержащий области с треугольниками 
res.append(k) # Добавление к массиву результатов число треугольников
for region in regionprops(label_image): # Начинаем обработку каждого из найденного контура
    if 550 >= region.area >= 150: # Проверка, аналогичная предыдущему циклу
        cr.append(img[region.bbox[0]:region.bbox[2],region.bbox[1]:region.bbox[3]]) # Добавляем в массив изображений области с каждым треугольником
        res.append([int(region.centroid[0]), int(region.centroid[1])]) # Добавляем в массив результатов центр каждой полученной области (центр треугольника)

        
        

k = 1
for img in cr: # беребор каждого изображения
    img = filters.sobel(img)  # применение фильра Собеля в каждой части изображения для улучшения видимости контуров

    thresh = sk.threshold_yen(img) # применение бинаризации Йена для улучшения видимости яркости в каждоой точке
    bw = img > thresh

    cleared = clear_border(bw)

    selem = disk(4.5) # определение примитива для дилатации
    bw = dilation(cleared, selem) # применение дилатации для объединения точек в один кластер

    selem = disk(1) # определение примитива для эрозии

    bw = erosion(bw, selem) # применение эрозии для удаления возможно появившихся элементов

    cleared = clear_border(bw)

    label_image = label(cleared)


    boxes = [] # массив для записи границ областей
    areas = [] # массив для записи площади областей для определения числа точек в нем
    dist = [] # матрица для записи расстояний между каждым кластером
    
    for region in regionprops(label_image): # анализ аналогичный пункту 1
        if  region.area >=150:
            minr, minc, maxr, maxc = region.bbox
            boxes.append(region.bbox) # запись координат области для вычисления расстояний между кластерами
            areas.append(region.area) # запись площади области для определения числа точек в нем
            
    for i in range(len(boxes)):
        temp = []
        for j in range(len(boxes)): # перебор всех возможных пар объектов
            l1 = boxes[i][0], boxes[i][1] 
            h1 = boxes[i][2], boxes[i][3]
            l2 = boxes[j][0], boxes[j][1]
            h2 = boxes[j][2], boxes[j][3] # определение границ первой и второй области
            temp.append(min(math.dist(l1,h2),math.dist(l1,l2),math.dist(h1,h2),math.dist(h1,l2))) # подсчет минимального расстояния между рамками объектов для выявления шумов на границах изображения
        dist.append(temp) # добавление полученного вектора расстояний в матрицу расстояний

    res[k].append(processing(areas, dist)) # обработка полученных наборов расстояний и площадей для каждого треугольника
    k += 1


In [12]:
f = open("result.txt", "w")
f.write(str(res[0]) + "\n")
for i in range(1, len(res)):
    str1 = str(res[i][0]) + ", " + str(res[i][1]) + "; " + str(res[i][2]) + "\n"
    str1 = str1.replace('(', "")
    str1 = str1.replace(')', "")
    f.write(str1)

In [2]:
def processing(a, d):
    if len(a) == 3:
        return(proc3(a))
    elif len(a) == 2:
        return(proc2(a))
    elif len(a) == 1:
        return(proc1(a))
    else:
        return(extraproc(a))

In [3]:
def proc3(a): # обработка полученных наборов расстояний и площадей для треугольников с кластерами из 3 элементов
    if a[0] <= 200 and a[1] <= 200 and a[2] >= 250:
        return 1, 1, 4
    elif a[0] <= 300 and 450 <= a[1] <= 500 and a[2] >= 250:
        return 1, 1, 1
    elif a[0] <= 300 and 450 <= a[1] <= 650 and a[2] <= 600:
        return 1, 3, 3
    elif a[0] <= 400 and 900 <= a[1] <= 1000 and 900 <= a[2] <= 1000:
        return 1, 5, 5
    elif a[0] <= 210 and a[1] <= 210 and a[2] >= 250:
        return 1, 1, 4
    elif a[0] <= 220 and a[1] <= 450 and a[2] <= 220:
        return 1, 1, 1
    elif a[0] <= 310 and a[1] <= 850 and a[2] <= 850:
        return 1, 5, 5
    elif 430 <= a[0] <= 440 and 300 <= a[1] <= 310 and 250 <= a[2] <= 300:
        return 1, 1, 3


In [4]:
def proc2(a): # обработка полученных наборов расстояний и площадей для треугольников с кластерами из 2 элементов
    if 240 <= a[0] <= 250 and a[1] <= 350:
        return 1, 1, 0
    elif a[0] <= 230 and a[1] <= 620:
        return 2, 0, 0
    elif 350 <= a[0] <= 380 and a[1] <= 250:
        return 1, 1, 0
    elif 500 <= a[0] <= 600 and a[1] <= 620:
        return 3, 0, 0
    elif 270 <= a[0] <= 300 and 270 <= a[1] <= 280:
        return 1, 3, 2
    elif 300 <= a[0] <= 310 and 700 <= a[1] <= 720:
        return 1, 3, 2
    elif 400 <= a[0] <= 450 and 200 <= a[1] <= 230:
        return 3, 3, 0
    elif 380 <= a[0] <= 390 and 350 <= a[1] <= 370:
        return 1, 2, 2
    elif 900 <= a[0] <= 1000 and 200 <= a[1] <= 250:
        return 1, 3, 4
    elif 430 <= a[0] <= 450 and 330 <= a[1] <= 350:
        return 2, 3, 4
    elif 300 <= a[0] <= 320 and 330 <= a[1] <= 350:
        return 2, 3, 3

In [5]:
def proc1(a): # обработка полученных наборов расстояний и площадей для треугольников с кластерами из 1 элемента
    if 450 <= a[0] <= 500:
        return 2, 0, 0
    elif 340 <= a[0] <= 350:
        return 2, 3, 3
    elif 650 <= a[0] <= 700:
        return 2, 3, 5
    elif 286 <= a[0] <= 300:
        return 3, 4, 4
    elif 280 <= a[0] <= 285:
        return 1, 1, 2
    elif 330 <= a[0] <= 340:
        return 1, 3, 4
    elif 1900 <= a[0]:
        return 1, 5, 5

In [6]:
def extraproc(a): # обработка экстренных случаев
    if len(a) == 0:
         return 3, 0, 0
    elif len(a) == 4:
        if a[0] <= 300 and a[1] <= 260 and a[2] <= 170 and a[3] <= 230:
             return 1, 3, 3  
        else:
             return 2, 3, 1